In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from matplotlib.ticker import StrMethodFormatter

#########################################################################################
# table 1.1 starts 
# Read data from files (companies and rounds2)
# Renamed file rounds2 to rounds as it was giving issue with name : round2
companies=pd.read_csv (r"D:\UPGRAD\assignments\companies.txt",sep="\t", encoding="ISO-8859-1")
rounds2=pd.read_csv (r"D:\UPGRAD\assignments\rounds.csv", encoding="ISO-8859-1")

# Fetch the uniqe companies from the companies 
compaines_temp=companies.apply(lambda x: x.astype(str).str.lower())
unique_companies=compaines_temp.drop_duplicates(subset='permalink',keep = False, inplace = False)
#print("Unique count of Companies: ",len(unique_companies['permalink']))


# Fetch the uniqe companies from the rounds 
rounds2_temp=rounds2.apply(lambda x: x.astype(str).str.lower())
unique_rounds=rounds2_temp.drop_duplicates(subset='company_permalink',keep = False, inplace = False) 
#print("Unique count of Companies in rounds data frame: ",len(unique_rounds['company_permalink']))

# code for looking that all companies from rounds2 are available in companies or not 
permalink_compaines=unique_companies['permalink']
company_permalink_rounds=unique_rounds['company_permalink']
#print(company_permalink_rounds.isin(permalink_compaines))

# code to merge both the data frame
master_frame=pd.merge(compaines_temp,rounds2_temp,left_on='permalink',right_on='company_permalink')
# creating a copy of merged data frame t obe used later
master_frame_original=pd.merge(compaines_temp,rounds2_temp,left_on='permalink',right_on='company_permalink')
#print("master_frame_original  ",master_frame.info())

# table 1.1 ends 
#######################################################################################

#table 2.1 starts

master_frame["raised_amount_usd"].replace({"nan": ""}, inplace=True)
master_frame['raised_amount_usd']=pd.to_numeric(master_frame['raised_amount_usd'])
mean_by_funding=master_frame.groupby(['funding_round_type'])['raised_amount_usd'].mean()
#mean_by_funding.head(20)
total_by_funding=master_frame.groupby(['funding_round_type'])['raised_amount_usd'].sum()
# plot graph between mean_by_funding and total_by_funding

index = ['venture', 'angel', 'seed','private_equity']
df = pd.DataFrame({'averagebyfunding': mean_by_funding}, index=index)
# it can be uncommented to see the graph 
#ax = df.plot.bar(rot=0)
df1 = pd.DataFrame({'totalamountbyfunding': total_by_funding}, index=index)
#ax = df1.plot.bar(rot=0)
# plot graph for fraction of total investments (globally) in angel,
# venture, seed, and private equity, and the average amount of investment in each funding type



#table 2.1 ends
######################################################################################
#table 3.1 starts
private_equity_filter = master_frame[master_frame['funding_round_type']=='private_equity']
group_countries=private_equity_filter.groupby('country_code')['raised_amount_usd'].sum()
top9Englishcountries=group_countries.nlargest(9)
# graph to plot top9 countries and total amount investment type 
top9Englishcountries.index.name = 'country_code'
# can be uncommneted later
#top9Englishcountries.plot.line()
#group_countries.plot(kind='bar')
top9Englishcountries.head(20)
#print(top9countries)
#table 3.1 ends
############################################################################################
#table 5.1 starts

# master data frame 'category_list' to lower case 
master_frame_original["raised_amount_usd"].replace({"nan": ""}, inplace=True)
master_frame_original['raised_amount_usd']=pd.to_numeric(master_frame['raised_amount_usd'])
master_frame_original['category_list']=master_frame_original['category_list'].str.lower()
# master data frame 'category_list' remove after |
master_frame_original['category_list']=master_frame_original['category_list'].str.split('|').str[0]
#master_frame_original.head()

# Read mapping csv 
mapping=pd.read_csv (r"D:\UPGRAD\assignments\mapping.csv", encoding="ISO-8859-1")
# lower case of "category_list" column
mapping['category_list']=mapping['category_list'].str.lower()

##############################function ##########

# Create new function to create new column:starts 
def label_sector (row):
   if row['Automotive & Sports'] == 1 :
      return 'Automotive & Sports'
   if row['Blanks'] == 1 :
      return 'Blanks'
   if row['Cleantech / Semiconductors'] == 1 :
       return 'Cleantech / Semiconductors'
   if row['Entertainment'] == 1 :
      return 'Entertainment'
   if row['Health'] == 1 :
      return 'Health'
   if row['Manufacturing'] == 1 :
      return 'Manufacturing'
   if row['News, Search and Messaging'] == 1 :
      return 'News, Search and Messaging'
   if row['Others'] == 1 :
      return 'Others'
   if row['Social, Finance, Analytics, Advertising'] == 1 :
      return 'Social, Finance, Analytics, Advertising'
 # Create new function to creae new column:ends 



####################################

# create new column of 'primary sector'
mapping['primary_sector'] = mapping.apply (lambda row: label_sector(row), axis=1)
# drop extra column
mapping=mapping.drop(columns=['Automotive & Sports','Social, Finance, Analytics, Advertising', 'Blanks','Cleantech / Semiconductors','Entertainment','Health','Manufacturing','News, Search and Messaging','Others'])
#mapping.info()
# merge data frame 
master_frame_original_merged=pd.merge(master_frame_original,mapping,left_on='category_list',right_on='category_list')
#print("master_frame_original_merged",master_frame_original_merged.info())
#master_frame_original_merged.info()
# remove unwanted column 
#master_frame_original_merged.head(100)

# D1 frame
D1=master_frame_original_merged[(master_frame_original_merged['country_code']=='usa') & 
                                (master_frame_original_merged['funding_round_type']=='private_equity')]
flag=D1["raised_amount_usd"].between(5000000, 15000000, inclusive = True) 
D1=D1[flag]

###############################################

# D2 frame 
D2=master_frame_original_merged.loc[(master_frame_original_merged['country_code']=='ind') & 
                                (master_frame_original_merged['funding_round_type']=='private_equity')]
flag=D2['raised_amount_usd'].between(5000000, 15000000, inclusive = True)
D2=D2[flag]
##############################################

D3=master_frame_original_merged.loc[(master_frame_original_merged['country_code']=='gbr') & 
                                (master_frame_original_merged['funding_round_type']=='private_equity')]
flag=D3['raised_amount_usd'].between(5000000, 15000000, inclusive = True)
D3=D3[flag]

###################################################################################
 #Total number of Investments (count)
print("Investment sectors in usa::::",D1.primary_sector.unique() )
print("Investment sectors in ind::::",D2.primary_sector.unique())
print("Investment sectors in gbr::::",D3.primary_sector.unique())
#############################################################################
# Total amount of Investments (dollars)

print("Total amount of inverstment in usa::::",D1['raised_amount_usd'].sum())
print("Total amount of inverstment in ind::::",D2['raised_amount_usd'].sum())
print("Total amount of inverstment in gbr::::",D3['raised_amount_usd'].sum())
############################################################################
# Top sector name , no. of investment wise 
#print("#####",D1.head(100))
print("no. of category wise investment under primary sector in usa",D1.groupby('primary_sector')['category_list'].count())
print("no. of category wise investmen under primary sector in ind",D2.groupby('primary_sector')['category_list'].count())
print("no. of category wise investmen under primary sector in gbr",D3.groupby('primary_sector')['category_list'].count())

# plot graph 3 
d1_temp=D1.groupby('primary_sector')['category_list'].count()
d1_temp.index.name = 'primary_sector'
#d1_temp.plot.line()
d2_temp=D2.groupby('primary_sector')['category_list'].count()
d2_temp.index.name = 'primary_sector'
#d2_temp.plot.line()
d3_temp=D3.groupby('primary_sector')['category_list'].count()
d3_temp.index.name = 'primary_sector'
d3_temp.plot.line()

###############################################################################

# Which country receives highest investment 
D1_temp=D1[D1['primary_sector']=='Cleantech / Semiconductors']
D1_temp1=D1_temp.sort_values('raised_amount_usd',ascending=False)
print("##Highest investment US company ###",D1_temp1.head(n=3))
# india as multiple in 1st group Cleantech / Semiconductors,Entertainment,Others
D2_temp=D2[D2['primary_sector'].isin(['Cleantech / Semiconductors','Entertainment','Others'])]
#.isin([3, 6])
D2_temp1=D2_temp.sort_values('raised_amount_usd',ascending=False)
print("##Highest investment IND company ###",D2_temp1.head(n=3))
# gbr as multiple in 1st group Social, Finance, Analytics, Advertising,Others, News, Search and Messaging, Manufacturing, Cleantech / Semiconductors
D3_temp=D3[D3['primary_sector'].isin(['Social, Finance, Analytics, Advertising','Advertising','Others','News, Search and Messaging','Manufacturing','Cleantech / Semiconductors'])]
D3_temp1=D3_temp.sort_values('raised_amount_usd',ascending=False)
print("##Highest investment GBR company ###",D3_temp1.head(n=3))

# Which country 2nd receives highest investment 
D1_temp2=D1[D1['primary_sector']=='Others']
D1_temp3=D1_temp2.sort_values('raised_amount_usd',ascending=False)
print("##Highest 2nd  investment US company ###",D1_temp3.head(n=3))

# India is all haivng 1 investment so no 2nd 

#print("##Highest investment IND company ###",D1.head(n=3))
# gbr as multiple in 1st group Social, Finance, Analytics, Advertising,Others, News, Search and Messaging, Manufacturing, Cleantech / Semiconductors
D3_temp2=D3[D3['primary_sector'].isin(['Automotive & Sports','Entertainment'])]
D3_temp3=D3_temp2.sort_values('raised_amount_usd',ascending=False)
print("##Highest 2nd investment GBR company ###",D3_temp3.head(n=2))



Investment sectors in usa:::: ['News, Search and Messaging' 'Others' 'Entertainment'
 'Cleantech / Semiconductors' 'Health'
 'Social, Finance, Analytics, Advertising' 'Manufacturing'
 'Automotive & Sports']
Investment sectors in ind:::: ['Others' 'Cleantech / Semiconductors' 'Entertainment']
Investment sectors in gbr:::: ['Others' 'Cleantech / Semiconductors' 'News, Search and Messaging'
 'Manufacturing' 'Entertainment' 'Automotive & Sports'
 'Social, Finance, Analytics, Advertising']
Total amount of inverstment in usa:::: 2620954330.0
Total amount of inverstment in ind:::: 31600000.0
Total amount of inverstment in gbr:::: 128775404.0
no. of category wise investment under primary sector in usa primary_sector
Automotive & Sports                         6
Cleantech / Semiconductors                 99
Entertainment                               7
Health                                     34
Manufacturing                              16
News, Search and Messaging                 31
Others